In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Droping unnecessary columns
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [4]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
#one hot encoding Geougraphy
from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder(sparse_output=False)
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
data = pd.concat([data.drop(['Geography'], axis=1), geo_encoded_df], axis=1)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
with open('label_encoder_gender.pkl', 'wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('onehot_encoder_geo.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_geo, f)

In [9]:
#Dividing the dataset into features and target variable

X = data.drop(['Exited'], axis=1)
y = data['Exited']

#Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Saving the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [11]:
#Build our ANN Model
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)), #hidden layer1 connected with input layer
        Dense(32, activation='relu'),  #hidden layer2
        Dense(1, activation='sigmoid'), #output layer
    ]
)

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
#Optimizer and loss function
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryCrossentropy()

In [14]:
#Compiling the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [15]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#Setting up callbacks
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)



In [18]:
#Early stopping to prevent overfitting
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [19]:
#Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorboard_callback, early_stopping_callback],
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3208 - accuracy: 0.8683 - val_loss: 0.3433 - val_accuracy: 0.8635
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3196 - accuracy: 0.8673 - val_loss: 0.3416 - val_accuracy: 0.8580
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3189 - accuracy: 0.8666 - val_loss: 0.3420 - val_accuracy: 0.8615
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3169 - accuracy: 0.8687 - val_loss: 0.3543 - val_accuracy: 0.8515
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3167 - accuracy: 0.8673 - val_loss: 0.3455 - val_accuracy: 0.8600
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3160 - accuracy: 0.8685 - val_loss: 0.3479 - val_accuracy: 0.8585
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3143 - accuracy: 0.8679 - val_loss: 0.3418 - val_accuracy: 0.8630

In [ ]:
model.save('model.h5')


d:\annclassification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
#load tensorboard extension
%load_ext tensorboard

In [24]:
%tensorboard --logdir logs/fit/20250525-034021

Reusing TensorBoard on port 6007 (pid 1088), started 0:01:07 ago. (Use '!kill 1088' to kill it.)